# Zonal Averaging

This section demonstrates how to perform Zonal Averaging using UXarray, covering both non-conservative and conservative methods.


In [ ]:
import numpy as np

import uxarray as ux

uxds = ux.open_dataset(
    "../../test/meshfiles/ugrid/outCSne30/outCSne30.ug",
    "../../test/meshfiles/ugrid/outCSne30/outCSne30_vortex.nc",
)
uxds["psi"].plot(cmap="inferno", periodic_elements="split")

## What is a Zonal Average/Mean?

A zonal average (or zonal mean) is a statistical measure that represents the average of a variable along one or more lines of constant latitude. In other words, it's the mean value calculated around the sphere at constant latitudes.

UXarray provides two types of zonal averaging:
- **Non-conservative**: Weights candidate faces by the length of intersection of a line of constant latitude
- **Conservative**: Preserves integral quantities by weighting faces by their area overlap with latitude bands

```{seealso}
[NCL Zonal Average](https://www.ncl.ucar.edu/Applications/zonal.shtml)
```

## Non-Conservative Zonal Averaging

The non-conservative method samples values at specific lines of constant latitude. This is the default behavior and is suitable for visualization and general analysis where exact conservation is not required.

In [ ]:
zonal_mean_psi = uxds["psi"].zonal_mean()
zonal_mean_psi

The default latitude range is between -90 and 90 degrees with a step size of 10 degrees. 

In [ ]:
(zonal_mean_psi.plot.line() * zonal_mean_psi.plot.scatter(color="red")).opts(
    title="Zonal Average Plot (Default)", xticks=np.arange(-90, 100, 20), xlim=(-95, 95)
)

The range of latitudes can be modified by using the `lat` parameter. It accepts:

* **Single scalar**: e.g., `lat=45`
* **List/array**: e.g., `lat=[10, 20]` or `lat=np.array([10, 20])`
* **Tuple**: e.g., `(min_lat, max_lat, step)`

In [ ]:
zonal_mean_psi_large = uxds["psi"].zonal_mean(lat=(-90, 90, 1))

In [ ]:
(
    zonal_mean_psi_large.plot.line()
    * zonal_mean_psi_large.plot.scatter(color="red", s=1)
).opts(
    title="Zonal Average Plot (Larger Sample)",
    xticks=np.arange(-90, 100, 20),
    xlim=(-95, 95),
)

## Conservative Zonal Averaging

Conservative zonal averaging preserves integral quantities (mass, energy, momentum) by computing area-weighted averages over latitude bands. This is essential for climate model analysis, energy budget calculations, and any application requiring physical conservation.

### Key Differences from Non-Conservative:
- **Non-conservative**: Samples at specific latitude lines
- **Conservative**: Averages over latitude bands between adjacent lines
- **Conservation**: Preserves global integrals to machine precision

In [ ]:
# Conservative zonal averaging with bands
bands = np.array([-90, -60, -30, 0, 30, 60, 90])
conservative_result = uxds["psi"].zonal_mean(lat=bands, conservative=True)
conservative_result

### Conservation Verification

A key advantage of conservative zonal averaging is that it preserves global integrals.

In [ ]:
# Test conservation property
global_mean = uxds["psi"].mean()
full_sphere_conservative = uxds["psi"].zonal_mean(lat=[-90, 90], conservative=True)
conservation_error = abs(global_mean.values - full_sphere_conservative.values[0])

print(f"Global mean: {global_mean.values:.12f}")
print(f"Conservative full sphere: {full_sphere_conservative.values[0]:.12f}")
print(f"Conservation error: {conservation_error:.2e}")

### Understanding the lat Parameter

Both conservative and non-conservative modes can use the same `lat` parameter, but they interpret it differently:

- **Non-conservative**: Creates sample points at the specified latitudes
- **Conservative**: Uses the latitudes as band edges, creating bands between adjacent points

In [ ]:
# Demonstrate signature behavior
lat_tuple = (-90, 90, 30)  # Every 30 degrees

# Non-conservative: samples at lines
non_cons_lines = uxds["psi"].zonal_mean(lat=lat_tuple)
print(f"Non-conservative with lat={lat_tuple}:")
print(f"Sample points: {non_cons_lines.coords['latitudes'].values}")
print(f"Count: {len(non_cons_lines.coords['latitudes'].values)} points")

# Conservative: creates bands between lines
cons_bands = uxds["psi"].zonal_mean(lat=lat_tuple, conservative=True)
print(f"\nConservative with lat={lat_tuple}:")
print(f"Band centers: {cons_bands.coords['latitudes'].values}")
print(f"Count: {len(cons_bands.coords['latitudes'].values)} bands")

### Visual Comparison: Conservative vs Non-ConservativeThe differences between methods reflect their fundamental approaches:- **Conservative**: More accurate for physical quantities because it accounts for the actual area of each face within latitude bands- **Non-conservative**: Faster but approximates by sampling at specific latitude linesThe differences you see indicate how much area-weighting matters for your specific data and grid resolution.

In [ ]:
import matplotlib.pyplot as plt

# Compare with non-conservative at same latitudes
band_centers = 0.5 * (bands[:-1] + bands[1:])
non_conservative_comparison = uxds["psi"].zonal_mean(lat=band_centers)

# Create comparison plot
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

ax.plot(
    conservative_result.coords["latitudes"],
    conservative_result.values,
    "o-",
    label="Conservative (bands)",
    linewidth=3,
    markersize=8,
    color="red",
)
ax.plot(
    non_conservative_comparison.coords["latitudes"],
    non_conservative_comparison.values,
    "s--",
    label="Non-conservative (lines)",
    linewidth=2,
    markersize=7,
    color="blue",
    alpha=0.7,
)

ax.set_xlabel("Latitude (degrees)")
ax.set_ylabel("Zonal Mean Value")
ax.set_title("Conservative vs Non-Conservative Zonal Averaging")
ax.grid(True, alpha=0.3)
ax.legend()
ax.set_xlim(-90, 90)

plt.tight_layout()
plt.show()

### Understanding the Differences

The differences between conservative and non-conservative results depend on several factors:

1. **Grid Resolution**: Higher resolution grids show smaller differences
2. **Data Variability**: Rapidly changing fields show larger differences
3. **Latitude Band Size**: Wider bands increase the importance of area-weighting

**Which is more accurate?**
- **Conservative**: More accurate for physical quantities (mass, energy, momentum) because it preserves integral properties
- **Non-conservative**: Adequate for visualization and qualitative analysis

**When differences matter most:**
- Variable resolution grids (where face sizes vary significantly)
- Physical conservation requirements
- Quantitative analysis and budget calculations

In [ ]:
# Quantify the differences
differences = conservative_result.values - non_conservative_comparison.values
max_diff = np.max(np.abs(differences))
mean_diff = np.mean(np.abs(differences))

print(f"Maximum absolute difference: {max_diff:.6f}")
print(f"Mean absolute difference: {mean_diff:.6f}")
print(
    f"Relative difference (max): {max_diff / np.mean(np.abs(conservative_result.values)) * 100:.3f}%"
)

## Combined Plots

It is often desired to plot the zonal average along side other plots, such as color or contour plots. 

In [ ]:
# Create combined plot with matplotlib
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Left: Data field
uxds["psi"].plot(ax=ax1, cmap="inferno")
ax1.set_title("Global Field")

# Right: Zonal average
ax2.plot(
    zonal_mean_psi.values,
    zonal_mean_psi.coords["latitudes"],
    "o-",
    linewidth=2,
    markersize=4,
)
ax2.set_xlabel("Zonal Mean Value")
ax2.set_ylabel("Latitude (degrees)")
ax2.set_title("Zonal Average")
ax2.grid(True, alpha=0.3)
ax2.set_ylim(-90, 90)
ax2.set_xlim(0.8, 1.2)

plt.suptitle("Combined Zonal Average & Raster Plot")
plt.tight_layout()
plt.show()